In [1]:
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import os

In [2]:
%pwd

'c:\\Users\\deyar\\OneDrive\\Desktop\\cnnClassifier\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\deyar\\OneDrive\\Desktop\\cnnClassifier'

In [5]:
from src.cnnClassifier.constants.cont import PARAMS_FILE_PATH,CONFIG_FILE_PATH
from src.cnnClassifier.logger import logging
from src.cnnClassifier.exception import customexception

In [6]:
import yaml
from box import ConfigBox

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEval:
    trained_model_path: Path
    EPOCHS: int
    LEARNING_RATE: float
    class_mode: str
    color_mode: str
    BATCH_SIZE: int

In [8]:
import sys

In [9]:
class ConfigManager:
    def __init__(self,paramspath=PARAMS_FILE_PATH,configpath=CONFIG_FILE_PATH):
        try:
         self.config=ConfigBox(yaml.safe_load(open(configpath)))
         self.params=ConfigBox(yaml.safe_load(open(paramspath)))
         logging.info("read yaml for model evaluation and mlflow")
        except Exception as e:
         logging.info("dir is not created")
         raise customexception(e,sys)
    def get_model_eval_config(self):
        param_config=self.params
        train=self.config.training
        model_eval_config=ModelEval(
            trained_model_path=train.trained_model_path,
            EPOCHS=param_config.EPOCHS,
            LEARNING_RATE=param_config.LEARNING_RATE,
            BATCH_SIZE=param_config.BATCH_SIZE,
            class_mode=param_config.class_mode,
            color_mode=param_config.color_mode)
        return model_eval_config

In [10]:
y=ConfigManager()

In [11]:
yy=y.get_model_eval_config()

In [12]:
yy

ModelEval(trained_model_path='artifacts/training/model.h5', EPOCHS=10000, LEARNING_RATE=0.001, class_mode='categorical', color_mode='rgb', BATCH_SIZE=16)

In [13]:
import dagshub
import mlflow

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

In [15]:
from tensorflow.keras.metrics import Precision, Recall

In [16]:
from urllib.parse import urlparse

In [ ]:
class getmodel__eval():
  def eval_result(self,p:ModelEval):
    self.model_path=(p.trained_model_path)
    self.EPOCHS=p.EPOCHS
    self.LEARNING_RATE=p.LEARNING_RATE
    self.class_mode=p.class_mode
    self.color_mode=p.color_mode
    self.BATCH_SIZE=p.BATCH_SIZE
    test_datagen=ImageDataGenerator(rescale=1/255) #normalize
    test_data_path = Path("artifacts") / "data_ingestion" / "Data" / "test"
    test_data = test_datagen.flow_from_directory(test_data_path,
    target_size=(224,224),
    color_mode=self.color_mode,
    class_mode=self.class_mode,  #train data generator
    batch_size=self.BATCH_SIZE)
    logging.info("data has been ready for model eval")
    model=load_model(self.model_path)
    logging.info("model has been  loaded for model eval")
    dagshub.auth.add_app_token('33fa4194997984f4200bf5a3b40710970754c0ce')
    dagshub.init(repo_owner='arijit99999', repo_name='CNNclassifier', mlflow=True)
    logging.info("dagshub has been  added")
    mlflow.set_registry_uri("https://dagshub.com/arijit99999/CNNclassifier.mlflow")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    print(tracking_url_type_store)
    with mlflow.start_run():
     self.eval=model.evaluate(test_data)
     mlflow.log_param('EPOCHS',self.EPOCHS)
     mlflow.log_param('LEARNING_RATE',self.LEARNING_RATE)
     mlflow.log_param('class_mode',self.class_mode)
     mlflow.log_param('BATCH_SIZE',self.BATCH_SIZE)
     mlflow.log_param('optimizer','RMSprop')
     mlflow.log_param('activation','relu')
     mlflow.log_metric("test_loss", self.eval[0])
     mlflow.log_metric("accuracy_score", self.eval[1])
     mlflow.log_metric("precision_score", self.eval[2])
     mlflow.log_metric("recall_score", self.eval[3])
     if tracking_url_type_store != "file":
              mlflow.keras.log_model(model, "model", registered_model_name="resnet50_model")
     else:
              mlflow.keras.log_model(model,"CNNmodel")
    mlflow.end_run()

In [27]:
evl=getmodel__eval()

In [28]:
evl.eval_result(yy)

Found 315 images belonging to 4 classes.


Initialized MLflow to track repo "arijit99999/CNNclassifier"

Repository arijit99999/CNNclassifier initialized!

https
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 581ms/step - accuracy: 0.4599 - loss: 1.4918 - precision: 0.8161 - recall: 0.2015


2025/05/10 08:27:15 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/10 08:27:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'resnet50_model'.
2025/05/10 08:28:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: resnet50_model, version 1
Created version '1' of model 'resnet50_model'.


🏃 View run carefree-shrew-760 at: https://dagshub.com/arijit99999/CNNclassifier.mlflow/#/experiments/0/runs/5776b58482ea4366a9ead6fedee004df
🧪 View experiment at: https://dagshub.com/arijit99999/CNNclassifier.mlflow/#/experiments/0
